<div style="font-size:18pt; padding-top:20px; text-align:center">Лабораторная работа 4. <b>Регрессия, классификация, кластеризация и
 </b> <span style="font-weight:bold; color:green">Spark/MLlib</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_hse@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Регрессия</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#1a">Линейная регрессия</a></li>
                <li><a href="#1b">Решающее дерево</a></li>
                <li><a href="#1c">Полимиальная регрессия</a></li>
            </ol>
        </li>
        <li><a href="#2">Классификация</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#2a">Логистическая регрессия</a></li>
                <li><a href="#2b">Решающее дерево</a></li>
                <li><a href="#2c">Полимиальная логистическая регрессия </a></li>
            </ol>
        </li>
        <li><a href="#3">Кластеризация</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#3a">K-Mean</a></li>
            </ol>
        </li>
        <li><a href="#4">Источники</a>
        </li>
    </ol>
</div>

<p><b>Инициализация</b></p>

In [ ]:
import pyspark

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.colors import ListedColormap
clrMap = ListedColormap(["blue", "red", "green"])

In [ ]:
from pyspark.mllib.regression import LabeledPoint

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Регрессия</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "1a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Линейная регрессия
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#1b">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.mllib.regression import LinearRegressionWithSGD

In [ ]:
sc = pyspark.SparkContext()

In [ ]:
data_rdd = sc.textFile("Datasets/Reg_A5.csv")

In [ ]:
def getLabeledPoint(line):
    if line[0]!="X":
        els = [float(el) for el in line.split(",")] 
        return LabeledPoint(els[1], els[0:1])

In [ ]:
data_xy_rdd = data_rdd.map(getLabeledPoint).filter(lambda x: x!=None)
data_xy_rdd.take(5)

Отображение исходных данных

In [ ]:
for el in data_xy_rdd.collect():
    plt.plot(el.features[0], el.label, "bo")
plt.grid(True)
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
plt.show()

Формирование обучающего и терировочного подмножеств

In [ ]:
train_rdd, test_rdd = data_xy_rdd.randomSplit([0.7, 0.3], seed=100)

In [ ]:
train_rdd.count(), test_rdd.count()

Обучение

In [ ]:
model_linReg = LinearRegressionWithSGD.train(train_rdd, intercept=True)
model_linReg

Проверка на тестовом подмножестве

In [ ]:
pairsTruePred_rdd = test_rdd.map(lambda x: (x.label, model_linReg.predict(x.features)))
pairsTruePred_rdd.take(5)

In [ ]:
mse = pairsTruePred_rdd.map(lambda x: (x[0]-x[1])**2).reduce(lambda x1, x2: x1+x2) / test_rdd.count()
mse

Построение итоговых графиков

In [ ]:
xx = np.arange(0, 5, 0.01)[np.newaxis,:]
plt.figure(1)
plt.subplot(1,1,1)

plt.plot(xx[0,:], model_linReg.predict(xx), c="black", label="", linewidth=2)
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
for el in data_xy_rdd.collect():
    plt.plot(el.features[0], el.label, "bo")
plt.grid(True)

plt.show()

<a name = "1b"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Решающее дерево
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1a">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#1c">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel

Обучение

In [ ]:
model_decTree = DecisionTree.trainRegressor(train_rdd, impurity="variance", maxDepth=2, categoricalFeaturesInfo={})
model_decTree

Тестирование

In [ ]:
test_pred_rdd = model_decTree.predict(test_rdd.map(lambda x: x.features))

In [ ]:
pairsTruePred_rdd = test_rdd.map(lambda x: x.label).zip(test_pred_rdd)
pairsTruePred_rdd.take(5)

In [ ]:
mse = pairsTruePred_rdd.map(lambda x: (x[0]-x[1])**2).reduce(lambda x1, x2: x1+x2) / test_rdd.count()
mse

Построение итоговых графиков

In [ ]:
#Построение итоговых графиков
xx = np.arange(0, 5, 0.01)
pr = [model_decTree.predict([el]) for el in xx]

plt.figure(1)
plt.subplot(1,1,1)
plt.plot(xx, pr, c="black", label="", linewidth=2)
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
for el in data_xy_rdd.collect():
    plt.plot(el.features[0], el.label, "bo")
plt.grid(True)

plt.show()

<a name = "1c"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            c. Полимиальная регрессия
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1b">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#2">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
#from sklearn.preprocessing import PolynomialFeatures
#from pyspark.mllib.feature import Normalizer
from pyspark.mllib.feature import StandardScaler

Формирование полинома + Нормализация

In [ ]:
deg = 10

In [ ]:
def get1DPolynomialFeatures(x, degree=deg):
    return [x**i for i in range(0, degree+1)]

In [ ]:
norm = StandardScaler(withMean=True, withStd=True)

In [ ]:
train_labels_rdd = train_rdd.map(lambda x: x.label)
train_features_poly_rdd = train_rdd.map(lambda x: get1DPolynomialFeatures(x.features[0], deg))

In [ ]:
train_features_poly_rdd.take(5)

In [ ]:
scaler = norm.fit(train_features_poly_rdd)

In [ ]:
train_features_poly_norm_rdd = scaler.transform(train_features_poly_rdd)
train_features_poly_norm_rdd.take(5)

In [ ]:
train_poly_rdd = train_labels_rdd.zip(train_features_poly_norm_rdd).map(lambda x: LabeledPoint(x[0], x[1]))
train_poly_rdd.take(5)

In [ ]:
test_labels_rdd = test_rdd.map(lambda x: x.label)
test_features_poly_rdd = test_rdd.map(lambda x: get1DPolynomialFeatures(x.features[0], deg))

In [ ]:
test_features_poly_norm_rdd = scaler.transform(test_features_poly_rdd)
test_features_poly_norm_rdd.take(5)

In [ ]:
test_poly_rdd = test_labels_rdd.zip(test_features_poly_norm_rdd).map(lambda x: LabeledPoint(x[0], x[1]))
test_poly_rdd.take(5)

Обучение

In [ ]:
model_linReg = LinearRegressionWithSGD.train(train_poly_rdd, intercept=True, regParam=10e-4, regType="l2", iterations=300)
model_linReg

Тестирование

In [ ]:
pairsTruePred_rdd = test_poly_rdd.map(lambda x: (x.label, model_linReg.predict(x.features)))
pairsTruePred_rdd.take(5)

In [ ]:
mse = pairsTruePred_rdd.map(lambda x: (x[0]-x[1])**2).reduce(lambda x1, x2: x1+x2) / test_rdd.count()
mse

Построение итоговых графиков

In [ ]:
dd = sc.parallelize(list(map(get1DPolynomialFeatures, np.arange(0, 5, 0.01))))
dd.take(5)

In [ ]:
ss=scaler.transform(dd)
ss.take(5)

In [ ]:
ff = ss.map(lambda x: model_linReg.predict(x))
yy = ff.collect()

In [ ]:
xx = np.arange(0, 5, 0.01)[np.newaxis,:]
plt.figure(1)
plt.subplot(1,1,1)

#xx1 = scaler.transform(np.array(list(map(get1DPolynomialFeatures, np.arange(0, 5, 0.01)))))

plt.plot(xx[0,:], yy, c="black", label="", linewidth=2)
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
for el in data_xy_rdd.collect():
    plt.plot(el.features[0], el.label, "bo")
plt.grid(True)

plt.show()

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Классификация</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "2a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Логистическая регрессия
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#2b">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithSGD

In [ ]:
#from pyspark.mllib.linalg import Vectors

In [ ]:
def getLabeledPoint(line):
    if line[0]!="X":
        els = [float(el) for el in line.split(",")]
        return LabeledPoint(els[2], els[:2])

In [ ]:
data_rdd = sc.textFile("Datasets/Cl_A5_V2.csv")

In [ ]:
data_xy_rdd = data_rdd.map(getLabeledPoint).filter(lambda x: x!= None)
data_xy_rdd.take(5)

In [ ]:
#Отображение исходных данных
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
for el in data_xy_rdd.collect():
    plt.plot(el.features[0], el.features[1], "o", color=clrMap.colors[int(el.label)])
plt.grid(True)
plt.show()

Формирование обучающего и терировочного подмножеств

In [ ]:
train_rdd, test_rdd = data_xy_rdd.randomSplit([0.7, 0.3], seed=0)

Обучение

In [ ]:
model_logReg = LogisticRegressionWithSGD.train(train_rdd, intercept=True)
model_logReg

Тестирование

In [ ]:
pairsTruePred_rdd = test_rdd.map(lambda x: (x.label, model_logReg.predict(x.features)))
pairsTruePred_rdd.take(5)

In [ ]:
accuracy = pairsTruePred_rdd.filter(lambda x: x[0] == x[1]).count() / test_rdd.count()
accuracy

In [ ]:
points

In [ ]:
step = 0.1
xx, yy = np.meshgrid(np.arange(-2, 14, step), np.arange(-4, 12, step))
points = np.c_[xx.ravel(), yy.ravel()]
Z = np.array(list(map(model_logReg.predict, points)))
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=[12, 4])

plt.subplot(1,2,1)
plt.title("Train data")
plt.contourf(xx, yy, Z, cmap=clrMap, alpha=.5)

for el in train_rdd.collect():
    plt.plot(el.features[0], el.features[1], "o", color=clrMap.colors[int(el.label)], markersize=8, alpha=0.5)
    plt.plot(el.features[0], el.features[1], "o", color=clrMap.colors[int(model_logReg.predict(el.features))], markersize=4)
    
plt.grid(True)

plt.subplot(1,2,2)
plt.title("Test data")
plt.contourf(xx, yy, Z, cmap=clrMap, alpha=.5)

for el in test_rdd.collect():
    plt.plot(el.features[0], el.features[1], "o", color=clrMap.colors[int(el.label)], markersize=8, alpha=0.5)
    plt.plot(el.features[0], el.features[1], "o", color=clrMap.colors[int(model_logReg.predict(el.features))], markersize=4)
plt.grid(True)

plt.show()

In [ ]:
model_logReg.save(sc, "logReg")
#model_logReg_reload = LogisticRegressionModel.load(sc, "logReg")

<a name = "2b"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Решающее дерево
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2a">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#2c">Далее</a>
            </div>
        </div>
    </div>
</div>

<a name = "2c"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            c. Полимиальная логистическая регрессия
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2b">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#3">Далее</a>
            </div>
        </div>
    </div>
</div>

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Кластеризация</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "3a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. K-Means
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#3b">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel

In [ ]:
def getArray(x):
    return np.array(x.split(","))

In [ ]:
data_rdd = sc.textFile("Datasets/Clusters.csv")
#data_rdd.take(5)

In [ ]:
data_arr_rdd = data_rdd.map(lambda x: getArray(x))
#data_arr_rdd.take(5)

In [ ]:
for el in data_arr_rdd.collect():
    plt.plot(el[0], el[1], "bo")
plt.grid(True)
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
plt.show()

Обучение

In [ ]:
clusters = KMeans.train(data_arr_rdd, 3, maxIterations=10, runs=10, initializationMode="k-means||")
clusters

In [ ]:
result_rdd = data_arr_rdd.map(lambda x: clusters.predict(x)).zip(data_arr_rdd)
result_rdd.take(5)

In [ ]:
for el in result_rdd.collect():
    plt.plot(el[1][0], el[1][1], "o", color=clrMap.colors[el[0]])
        
plt.grid(True)
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
plt.show()

In [ ]:
clusters.save(sc, "clusterModel")
sameModel = KMeansModel.load(sc, "clusterModel")

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a href="http://spark.apache.org/docs/latest/ml-guide.html">Machine Learning Library (MLlib) Guide</a>